In [1]:
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import xgboost as xgb
import elm
from imblearn.over_sampling import SMOTE 

import dataloader
import bandpower
from plot_confusion_matrix import plot_confusion_matrix


# Leave-one-subject out cross validation

In [2]:
for label_format in range(1,4):

    X, Y, df_all = dataloader.read_data(label_format=label_format)

    # Transform into power
    low, high = list(range(1,50)), list(range(2,51))
    powers, _, _ = bandpower.get_bandpower(X, 1000, low=low, high=high, dB_scale=True)

    # Select models
    model_names = ['svm', 'xgboost']
    sampling_names = ['original', 'SMOTE']
    cv_modes = ['KFold', 'LOSO']

    for model_name, sampling_name, cv_mode in itertools.product(model_names, sampling_names, cv_modes):

        file_name = 'label%d_%s_%s_%s.csv'%(label_format, model_name, sampling_name, cv_mode)
        description = 'Label: %d, Model: %s, Sampling: %s, CV mode: %s'%(label_format, model_name, sampling_name, cv_mode)

        # Average band power over channels
        num_sample = len(X)
        X_power = np.zeros((num_sample, len(low)))
        for i in range(len(X)):
            X_power[i,:] = np.mean(powers[i],0)

        kf = KFold(n_splits=10, shuffle=True, random_state=23)
        S = df_all['subject'].values
        folds = kf.split(X_power) if cv_mode=='KFold' else np.unique(S)

        train_acc_list = []
        val_acc_list = []
        train_recall_list = []
        val_recall_list = []
        train_prec_list = []
        val_prec_list = []
        num_val_list = []
        chance_list = []
        fold_name_list = []

        print('Sub  \t Chance\t | Train | Val')
        for i_fold, fold in enumerate(folds):
            if cv_mode == 'KFold':
                train_indices = fold[0]
                val_indices = fold[1]
            else:
                subID = fold
                train_indices = np.where(S!=subID)[0]
                val_indices = np.where(S==subID)[0]

            X_train, Y_train = X_power[train_indices,...], Y[train_indices]
            X_val, Y_val = X_power[val_indices,...], Y[val_indices]

            # Resample training data
            if sampling_name == 'SMOTE':
                sm = SMOTE(random_state=23)
                X_train, Y_train = sm.fit_resample(X_train, Y_train)

            # Train classifier
            if model_name in ['svm','xgboost']:
                if model_name == 'svm':
                    clf = svm.SVC()
                elif model_name == 'xgboost':
                    clf = xgb.XGBClassifier()
                clf.fit(X_train, Y_train)

                # Test classifier
                train_acc = clf.score(X_train, Y_train)
                val_acc = clf.score(X_val, Y_val)
                
                pred_train = clf.predict(X_train)
                pred_val = clf.predict(X_val)
                
                train_recall = recall_score(Y_train, pred_train)
                val_recall = recall_score(Y_val, pred_val)
                
                train_prec = precision_score(Y_train, pred_train)
                val_prec = precision_score(Y_val, pred_val)
                
            fold_name = 'Fold %d'%(i_fold) if cv_mode == 'KFold' else 'Sub %d'%(subID)
            chance = np.sum(Y_train)/len(Y_train)
            chance = (1-chance) if chance<0.5 else chance

            train_acc_list.append(train_acc)
            val_acc_list.append(val_acc)
            train_recall_list.append(train_recall)
            val_recall_list.append(val_recall)
            train_prec_list.append(train_prec)
            val_prec_list.append(val_prec)
            num_val_list.append(len(Y_val))
            fold_name_list.append(fold_name)
            chance_list.append(chance)

            print('%s\t %.1f%%\t | %.1f%% | %.1f%%'%(fold_name, chance*100, train_acc*100, val_acc*100))
            
            fileName_cm = './results/' + file_name[:file_name.find('.')] + '_' + fold_name
            plot_confusion_matrix(Y_train, pred_train, np.array(['Normal','Increase']), 
                                 fileName='%s_train.png'%(fileName_cm), title=fold_name)
            plot_confusion_matrix(Y_val, pred_val, np.array(['Normal','Increase']), 
                                 fileName='%s_val.png'%(fileName_cm), title=fold_name)

        avg_acc = sum([val_acc_list[i]*num_val_list[i] for i in range(len(num_val_list))])/num_sample
        avg_recall = sum([val_recall_list[i]*num_val_list[i] for i in range(len(num_val_list))])/num_sample
        avg_prec = sum([val_prec_list[i]*num_val_list[i] for i in range(len(num_val_list))])/num_sample
        print('Average val acc: %.1f%%'%(avg_acc*100))

        # Save result as csv file
        df_result = pd.DataFrame({description: fold_name_list, 'Chance_val_acc': chance_list, 
                                  'Train_acc': train_acc_list, 'Val_acc': val_acc_list,
                                  'Train_recall' : train_recall_list, 'Val_recall': val_recall_list,
                                  'Train_precision' : train_prec_list, 'Val_precision': val_prec_list})
        df_result.loc[len(df_result),description] = 'Average'
        df_result.loc[len(df_result)-1, 'Val_acc'] = avg_acc
        df_result.loc[len(df_result)-1, 'Val_recall'] = avg_recall
        df_result.loc[len(df_result)-1, 'Val_precision'] = avg_prec
        df_result.to_csv('./results/%s'%(file_name))

Load data from .mat files...


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:495: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val = np.array(val, copy=False)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Calculating the bandpower of time-series data...
freqs:  [0.000e+00 2.000e-01 4.000e-01 ... 4.996e+02 4.998e+02 5.000e+02]
Sub  	 Chance	 | Train | Val


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 0	 88.0%	 | 88.9% | 92.3%
Confusion matrix, without normalization
[[103   0]
 [ 13   1]]
Confusion matrix, without normalization
[[12  0]
 [ 1  0]]


...

Sub 22	 50.0%	 | 99.5% | 80.0%
Confusion matrix, without normalization
[[94  0]
 [ 1 93]]
Confusion matrix, without normalization
[[8 2]
 [0 0]]


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sub 23	 50.0%	 | 99.5% | 80.0%
Confusion matrix, without normalization
[[99  0]
 [ 1 98]]
Confusion matrix, without normalization
[[4 1]
 [0 0]]
Sub 24	 50.0%	 | 99.5% | 66.7%
Confusion matrix, without normalization
[[99  0]
 [ 1 98]]
Confusion matrix, without normalization
[[3 2]
 [0 1]]


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sub 25	 50.0%	 | 99.5% | 83.3%
Confusion matrix, without normalization
[[98  0]
 [ 1 97]]
Confusion matrix, without normalization
[[5 1]
 [0 0]]
Sub 26	 50.0%	 | 99.5% | 57.1%
Confusion matrix, without normalization
[[101   0]
 [  1 100]]
Confusion matrix, without normalization
[[2 1]
 [2 2]]
Average val acc: 61.5%
